In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## This notebook is a quick modeling for baseline submission.

In [ ]:
from fastai.text.all import *

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
print('Size of train data is ',train.shape)
train.head(1)

In [ ]:
text_lm = DataBlock(blocks=TextBlock.from_df('excerpt', is_lm=True),
                    get_x=ColReader('text'),
                    splitter=RandomSplitter())

dls_lm = text_lm.dataloaders(train, bs=64, seq_len=72)
dls_lm.show_batch(max_n=1)

In [ ]:
text_clas = DataBlock(blocks=(TextBlock.from_df('excerpt', vocab=dls_lm.vocab), RegressionBlock),
                      get_x=ColReader('text'),
                      get_y=ColReader('target'),
                      splitter=RandomSplitter())

dls = text_clas.dataloaders(train, bs=64)
dls.show_batch(max_n=1)

In [ ]:
# Commenting for submission purpose

# learner = text_classifier_learner(dls, AWD_LSTM, metrics=rmse)
# learner.fit_one_cycle(5, 2e-2)
# learner.export('model_lstm')

In [ ]:
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test.head(1)

In [ ]:
learner = load_learner('/kaggle/input/commonlitmodel/model_lstm')

In [ ]:
tdl = learner.dls.test_dl(test.excerpt)
preds, _ = learner.get_preds(dl=tdl)

In [ ]:
submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submission['target'] = preds.numpy()
submission.to_csv('submission.csv',index=False)